In [1]:
import pandas as pd
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

os.listdir()

jobs = pd.read_excel('jobsandschools.xlsx', sheet_name=0).set_index(
    'Unnamed: 0').fillna(0).replace('X', 1).T

schools = pd.read_excel('jobsandschools.xlsx', sheet_name=1).set_index(
    'Unnamed: 0').fillna(0).replace('X', 1).T


def generate_kmeans_curve(data, name):

    Sum_of_squared_distances = []
    K = range(1,6)
    for k in K:
        km = KMeans(n_clusters=k)
        km = km.fit(data)
        Sum_of_squared_distances.append(km.inertia_)

    plt.plot(K, Sum_of_squared_distances, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Sum_of_squared_distances')
    plt.title('Elbow Method For Optimal k')
    plt.savefig('kmeans_elbowplot_' + name + '.jpg')
    plt.show()

slikej = schools.copy()
slikej.columns = jobs.columns
both = pd.concat((jobs, slikej), axis=0)

generate_kmeans_curve(schools, 'schools')

generate_kmeans_curve(jobs, 'jobs')

generate_kmeans_curve(both, 'both')

def fit_pca(data, model):
    output = model(n_components = 2).fit_transform(data)
    #output = model.transform(data)
    return pd.DataFrame(output, index = data.index)
    

def scatter(data, name, cluster_assignments):
    ax = data.plot.scatter(0, 1, c = [{0:'red', 1:'blue', 2:'green', 3:'purple'}[c] for c in cluster_assignments])
    for i, txt in enumerate(data.index):
        ax.annotate(txt, (data.iloc[i, 0], data.iloc[i, 1]))
    plt.title(name)
    plt.savefig(name + '.png')
    plt.show()

for raw_data, source_name, n_clusters in zip([jobs, schools, both], ['jobs', 'schools', 'both'], [2, 3, 4]):
    cluster_assignments = KMeans(n_clusters=n_clusters).fit_predict(raw_data)
    for model, modelname in zip([PCA, TSNE], ['PCA', 'TSNE']):
        data = fit_pca(raw_data, model)
        scatter(data, source_name + '_' + modelname + '_' + str(max(cluster_assignments) + 1) + 'Clusters',
                cluster_assignments)

needed = pd.Series(jobs.mean(0).values - schools.mean(0).values, index = schools.columns)

norm_need = (needed - needed.min())/(needed.max()- needed.min()) * 2 - 1

needed = needed.to_frame()

needed.columns = ['need']

needed['jobs'] = jobs.mean(0).values
needed['schools'] = schools.mean(0).values



def highlight_greaterthan(s, threshold, column):
    is_max = pd.Series(data=False, index=s.index)
    return ['background-color: lightcoral' if s[column] > 0.1 else 
            'background-color: lightgreen' if s[column] < -.1 else ''
            for v in is_max]


needed = needed.round(2)

needed = needed[['jobs', 'schools', 'need']]

needed.index.name = None

# needed.style.apply(highlight_greaterthan, threshold=0, column='need', axis=1).to_excel('need_summarized.xlsx')

import pandas as pd

# Create a Pandas dataframe from some data.
# def sim_data():
#     df = pd.DataFrame(columns = ['id', 'name', 'age'])
#     df.loc[len(df), :] = [1, 'John', 21]
#     df.loc[len(df), :] = [2, 'Mary', -1]
#     df.loc[len(df), :] = [3, 'Ann', 0.01]
#     df.loc[len(df), :] = [4, 'Ben', .01]
#     return df
# sim = sim_data()
def write_with_color(df, filename):
    
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    red = workbook.add_format({'bg_color': '#FFC7CE'})
    green = workbook.add_format({'bg_color': '#CCFFCC'})

    start_row = 1
    start_col = 3
    end_row = len(df)
    end_cold = start_col

    worksheet.conditional_format(start_row, start_col, end_row, end_cold,
                                 {'type':     'cell',
                                  'criteria': '>',
                                  'value':    .1,
                                  'format':   green})

    worksheet.conditional_format(start_row, start_col, end_row, end_cold,
                                 {'type':     'cell',
                                  'criteria': '<',
                                  'value':    -.1,
                                  'format':   red})
    writer.save()
write_with_color(needed, 'needed.xlsx')

jobs.corr().dropna(how='all').dropna(how='all', axis=1).round(2)







<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Unnamed: 0,Mentions Python/Scripting languages,Mentions SQL/database languages,Mentions Big data technologies,Engineering Required,Engineering as a core focus,Modern Machine Learning math/research,Problem solving using machine learning,Business Communication/ interaction
Unnamed: 0,,,,,,,,
Mentions Python/Scripting languages,1.00,0.71,-0.25,0.25,0.50,0.50,0.25,-0.32
Mentions SQL/database languages,0.71,1.00,-0.71,0.00,0.71,0.00,0.00,-0.45
Mentions Big data technologies,-0.25,-0.71,1.00,-0.25,-0.50,0.25,0.50,0.32
Engineering Required,0.25,0.00,-0.25,1.00,0.50,0.50,-0.50,-0.32
Engineering as a core focus,0.50,0.71,-0.50,0.50,1.00,0.25,-0.25,-0.63
Modern Machine Learning math/research,0.50,0.00,0.25,0.50,0.25,1.00,0.50,-0.63
Problem solving using machine learning,0.25,0.00,0.50,-0.50,-0.25,0.50,1.00,-0.32
Business Communication/ interaction,-0.32,-0.45,0.32,-0.32,-0.63,-0.63,-0.32,1.00


In [2]:
schools.T.corr().round(2)

,NYU,CMU - Systems,CMU - Analytics,CMU - Human Centered,Northwestern,MIT,Stanford,Berkeley
NYU,1.00,0.25,0.82,0.00,0.41,0.00,0.41,0.27
CMU - Systems,0.25,1.00,0.00,-0.41,-0.41,0.00,0.61,-0.27
CMU - Analytics,0.82,0.00,1.00,0.20,0.50,-0.20,0.00,0.33
CMU - Human Centered,0.00,-0.41,0.20,1.00,0.50,0.60,-0.50,0.33
Northwestern,0.41,-0.41,0.50,0.50,1.00,0.50,-0.25,-0.17
MIT,0.00,0.00,-0.20,0.60,0.50,1.00,0.00,-0.33
Stanford,0.41,0.61,0.00,-0.50,-0.25,0.00,1.00,-0.17
Berkeley,0.27,-0.27,0.33,0.33,-0.17,-0.33,-0.17,1.00


In [3]:
jobs.T.corr().round(2)

,Google,Amazon,BofA,Two Sigma,Aetna,Coca-Cola
Google,1.00,0.10,0.22,-0.25,-0.50,-0.10
Amazon,0.10,1.00,0.36,0.10,0.41,0.25
BofA,0.22,0.36,1.00,-0.33,-0.22,0.53
Two Sigma,-0.25,0.10,-0.33,1.00,0.50,0.41
Aetna,-0.50,0.41,-0.22,0.50,1.00,0.41
Coca-Cola,-0.10,0.25,0.53,0.41,0.41,1.00


In [7]:
df_survey = pd.read_csv('Untitled form.csv').iloc[:,1:]

In [9]:
jobs.head()

Unnamed: 0,Mentions Python/Scripting languages,Mentions SQL/database languages,Mentions Big data technologies,Engineering Required,Engineering as a core focus,Traditional Machine Learning math/statistics,Modern Machine Learning math/research,Business Problem Solving,Problem solving using machine learning,Business Communication/ interaction
Google,1,1,0,1,1,1,1,1,1,0
Amazon,1,1,0,0,0,1,0,1,1,1
BofA,1,1,0,1,1,1,0,1,0,1
Two Sigma,1,0,1,1,0,1,1,1,1,1
Aetna,0,0,1,0,0,1,0,1,1,1


In [11]:
data_indexes = [0, 4, 1, 5, 7, 2, -1]

In [18]:
{s:j for j, s in zip(jobs_newcols.columns, survey_importance.columns)}

{'1. How important is Python programming to your job?': 'Mentions Python/Scripting languages',
 '2. How important is collaborative software development and deployment to your job?': 'Engineering as a core focus',
 '3. How important is data engineering to your job?': 'Mentions SQL/database languages',
 '4. How important is model development (ML/Deep Learning/stats) to your job?': 'Traditional Machine Learning math/statistics',
 '5. How important is KPI management to your job?': 'Business Problem Solving',
 '6. How important is business and data strategy to your job?': 'Mentions Big data technologies',
 '7. How important is communication to your job?': 'Business Communication/ interaction'}

In [15]:
jobs_newcols = jobs.iloc[:,data_indexes]
jobs_newcols.head(1)

Unnamed: 0,Mentions Python/Scripting languages,Engineering as a core focus,Mentions SQL/database languages,Traditional Machine Learning math/statistics,Business Problem Solving,Mentions Big data technologies,Business Communication/ interaction
Google,1,1,1,1,1,0,0


In [16]:
survey_importance = df_survey.iloc[:,::2]

In [4]:
os.listdir()

['leetcode.ipynb',
 'jobs_PCA_2Clusters.png',
 'both_PCA_4Clusters.png',
 '.DS_Store',
 'schools_TSNE_3Clusters.png',
 'jobsandschools.xlsx',
 'jobs_TSNE_2Clusters.png',
 'Untitled form.csv',
 'schools_PCA_3Clusters.png',
 'kmeans_elbowplot_both.jpg',
 'kmeans_elbowplot_schools.jpg',
 'kmeans_elbowplot_jobs.jpg',
 '.ipynb_checkpoints',
 'analyzejobsandschools.ipynb',
 '.git',
 'analyzejobsandschoolsold.ipynb',
 'both_TSNE_4Clusters.png',
 'Untitled form.csv.zip',
 'needed.xlsx']